In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

import dataset
assert tf.config.list_physical_devices('GPU')

Using TensorFlow backend.


In [2]:
MAXLEN = 100
BATCH_SIZE = 32
HIDE_LETTERS = False

def build_model():
    EMBED_DIM = 512
    UNITS = 256

    LETTERS_SIZE = len(dataset.letters_table)
    NIQQUD_SIZE = len(dataset.niqqud_table)
    DAGESH_SIZE = len(dataset.dagesh_table)
    SIN_SIZE = len(dataset.sin_table)

    common_input = tf.keras.Input(batch_shape=(None, MAXLEN), batch_size=BATCH_SIZE)
    
    common = layers.Embedding(LETTERS_SIZE, EMBED_DIM, mask_zero=True)(common_input)
    common = layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True, dropout=0.1), merge_mode='sum')(common)
    common = layers.add([common,
                         layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True, dropout=0.1), merge_mode='sum')(common)])

    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(common)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(common)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(common))
    ]
    if HIDE_LETTERS:
        outputs.append(layers.Softmax(name='C')(layers.Dense(LETTERS_SIZE)(common)))
        
    model = tf.keras.Model(inputs=[common_input], outputs=outputs)

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    tf.keras.utils.plot_model(model, to_file='model.png')
    model.summary()
    return model
    
model = build_model()
model.save_weights('./checkpoint_uninit')

def add_noise(original):
    noised = np.copy(original)
    indices = np.random.choice(np.arange(noised.size), replace=False, size=int(noised.size / MAXLEN))
    indices = np.unravel_index(indices, noised.shape)
    noised[indices] = 1  # Cannot be 0 since masking means it will not be considered as loss
    return noised

def fit(data, learning_rates):
    x  = data.normalized_texts
    vx = data.normalized_validation
    y  = {'N': data.niqqud_texts,      'D': data.dagesh_texts,      'S': data.sin_texts,      'C': data.normalized_texts     }
    vy = {'N': data.niqqud_validation, 'D': data.dagesh_validation, 'S': data.sin_validation, 'C': data.normalized_validation}
    if HIDE_LETTERS:
        x  = add_noise(x)
        vx = add_noise(vx)
        y['C'] = data.normalized_texts
        vy['C'] = data.normalized_validation
    return model.fit(x, y, validation_data=(vx, vy), batch_size=BATCH_SIZE, epochs=len(learning_rates),
          callbacks=[
              tf.keras.callbacks.LearningRateScheduler(lambda epoch, lr: learning_rates[epoch], verbose=0),
              # tf.keras.callbacks.ModelCheckpoint(filepath='checkpoints/ckpt_{epoch}', save_weights_only=True),
          ]
    )

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 512)     22528       input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 100, 256)     1574912     embedding[0][0]                  
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 100, 256)     1050624     bidirectional[0][0]              
________

In [3]:
def load_data(source):
    filenames = [os.path.join('texts', f) for f in source]
    return dataset.load_file(filenames, BATCH_SIZE, 0.1, maxlen=MAXLEN, shuffle=True)

In [22]:
rabanit = ['birkat_hamazon.txt', 'kuzari.txt', 'hakdama_leorot.txt', 'hartzaat_harav.txt', 'orhot_hayim.txt', 'rambam_mamre.txt', 'short_table.txt',
           'tomer_dvora.txt', 'elef_layla.txt', 'bialik', 'breslev.txt']

data_rabanit = load_data(rabanit)

In [23]:
modern = ['itzhak_berkman', 'zevi_scharfstein', 'pesah_kaplan', 'abraham_regelson', 'elisha_porat', 'uriel_ofek', 'yisrael_dushman', 'zvi_zviri',
          'sipurim.txt' ,'atar_hashabat.txt', 'ali_baba.txt', 'ricky.txt', 'imagination.txt', 'adamtsair.txt', 'katarsis.txt']

data_modern = load_data(modern)

In [28]:
model.load_weights('./checkpoint_uninit')
history = fit(data_rabanit, [3e-3, 3e-4])
model.save_weights('./checkpoint_rabanit')

Train on 66729 samples, validate on 7415 samples
Epoch 1/2
66729/66729 [==============================] - 116s 2ms/sample - loss: 0.4150 - N_loss: 0.3210 - D_loss: 0.0830 - S_loss: 0.0109 - N_accuracy: 0.8916 - D_accuracy: 0.9691 - S_accuracy: 0.9967 - val_loss: 0.2380 - val_N_loss: 0.1830 - val_D_loss: 0.0500 - val_S_loss: 0.0050 - val_N_accuracy: 0.9386 - val_D_accuracy: 0.9814 - val_S_accuracy: 0.9987
Epoch 2/2
66729/66729 [==============================] - 117s 2ms/sample - loss: 0.2150 - N_loss: 0.1638 - D_loss: 0.0465 - S_loss: 0.0047 - N_accuracy: 0.9452 - D_accuracy: 0.9829 - S_accuracy: 0.9988 - val_loss: 0.2005 - val_N_loss: 0.1529 - val_D_loss: 0.0433 - val_S_loss: 0.0043 - val_N_accuracy: 0.9493 - val_D_accuracy: 0.9842 - val_S_accuracy: 0.9989


In [29]:
model.load_weights('./checkpoint_rabanit')
history = fit(data_modern, [3e-3, 3e-4])

Train on 14688 samples, validate on 1632 samples
Epoch 1/2
14688/14688 [==============================] - 24s 2ms/sample - loss: 0.2952 - N_loss: 0.2213 - D_loss: 0.0683 - S_loss: 0.0056 - N_accuracy: 0.9269 - D_accuracy: 0.9712 - S_accuracy: 0.9984 - val_loss: 0.2544 - val_N_loss: 0.1882 - val_D_loss: 0.0607 - val_S_loss: 0.0055 - val_N_accuracy: 0.9385 - val_D_accuracy: 0.9745 - val_S_accuracy: 0.9986
Epoch 2/2
14688/14688 [==============================] - 24s 2ms/sample - loss: 0.2195 - N_loss: 0.1589 - D_loss: 0.0561 - S_loss: 0.0045 - N_accuracy: 0.9485 - D_accuracy: 0.9761 - S_accuracy: 0.9988 - val_loss: 0.2223 - val_N_loss: 0.1607 - val_D_loss: 0.0565 - val_S_loss: 0.0051 - val_N_accuracy: 0.9488 - val_D_accuracy: 0.9763 - val_S_accuracy: 0.9987curacy: 0.9761 - S


In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=2, ncols=2)

for n, v in enumerate(['accuracy', 'loss'], 0):
    for n1, t in enumerate(['D', 'N'], 0):
        p = ax[n][n1]
        p.plot(history.history[t + '_' + v][0:])
        p.plot(history.history['val_' + t + '_' +  v][0:])
        p.legend([t + '_Train', t + '_Test'], loc='center right')

plt.tight_layout()

In [31]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, '.')

In [30]:
def print_predictions(data, k):
    s = slice(k*BATCH_SIZE, (k+1)*BATCH_SIZE)
    batch = data.normalized_validation[s]
    prediction = model.predict(batch)
    [actual_niqqud, actual_dagesh, actual_sin] = [dataset.from_categorical(prediction[0]), dataset.from_categorical(prediction[1]), dataset.from_categorical(prediction[2])]
    [expected_niqqud, expected_dagesh, expected_sin] = [data.niqqud_validation[s], data.dagesh_validation[s], data.sin_validation[s]]
    actual = data.merge(batch, ns=actual_niqqud, ds=actual_dagesh, ss=actual_sin)
    expected = data.merge(batch, ns=expected_niqqud, ds=expected_dagesh, ss=expected_sin)
    for i, (a, e) in enumerate(zip(actual, expected)):
        print('מצוי: ', a)
        print('רצוי: ', e)
        print()

print_predictions(data_modern, 1)

מצוי:  אָבִיהָ מִכְתָּב דָּחוֹף מֵהַרְבִּי מִלְּיוּבָאוּוִיטְשׁ זְצוּקָ"ל, בּוֹ נִתְבַּקֵּשׁ אָבִיהָ לִנְסֹעַ מִיָּד לְטִסְמָנֶיָה. אָבִיהָ הִתְפַּלֵּא מַה יֵשׁ לוֹ לַעֲשׂוֹת
רצוי:  אָבִיהָ מִכְתָּב דָּחוּף מֵהָרַבִּי מִלְיוּבָּאווִיטְשׁ זֶצוּקָ"ל, בּוֹ נִתְבַּקֵּשׁ אָבִיהָ לִנְסֹעַ מִיָּד לְטַסְמַנְיָה. אָבִיהָ הִתְפַּלֵּא מַה יֵשׁ לוֹ לַעֲשׂוֹת

מצוי:  לָעִנְיָן. "אֵינִי יוֹדֵעַ," עָנָה סְמִית. "כָּל בֹּקֶר הֲרֵינִי עוֹשֶׂה סִבּוּב בִּמְכוֹנִית עַד חֲרָשַׁת-הַגּוּיִים, וְחוֹזֵר דֶּרֶךְ מִגְּדוֹ,
רצוי:  לָעִנְיָן. "אֵינִי יוֹדֵעַ," עָנָה סְמִית. "כָּל בֹּקֶר הֲרֵינִי עוֹשֶׂה סִבּוּב בַּמְּכוֹנִית עַד חֲרשֶׁת-הַגּוֹיִים, וְחוֹזֵר דֶּרֶךְ מְגִדּוֹ,

מצוי:  וְהַפָּרָה אַף הִיא הָפְכָה פָּנֶיהָ וְרָצָה לְגוֹנֵן עַל בַּר-בַּטְּנָה. נִשְׁתַּלְּטָה הֲפּוּגָה, וְיוּבַל קָרָא לְהִסּוּג, כִּי יֵשׁ לוֹ לְהַגִּיד מַשֶׁהוּ.
רצוי:  וְהַפָּרָה אַף הִיא הָפְכָה פָּנֶיהָ וְרָצָה לְגוֹנֵן עַל בַּר-בִּטְנָהּ. נִשְׁתַּלְּטָה הֲפוּגָה, וְיוּבַל קָרָא לְהִסּוֹג, כִּי יֵשׁ לוֹ לְהַגִּיד מַשֶׁהוּ.

מצוי:  יֵ